### **Что важно учитывать при использовании `GridSearchCV`?**

1. **Выбор гиперпараметров:**
   - Определите, какие гиперпараметры вы хотите оптимизировать. Например, для `RandomForestClassifier` это могут быть `n_estimators`, `max_depth`, `min_samples_split`.
   - Не перегружайте сетку слишком большим количеством гиперпараметров, иначе поиск станет очень долгим.

2. **Размер сетки параметров:**
   - Убедитесь, что сетка покрывает разумный диапазон значений гиперпараметров.
   - Для некоторых параметров (например, `learning_rate`) лучше использовать логарифмическую шкалу.

3. **Кросс-валидация (`cv`):**
   - Определяет, на сколько частей делится выборка (например, `cv=5` означает 5 фолдов).
   - Используйте стратифицированную кросс-валидацию (`StratifiedKFold`) для несбалансированных данных.

4. **Метрика:**
   - Укажите метрику для оптимизации (например, `accuracy`, `roc_auc`, `neg_mean_squared_error`).
   - Метрика должна соответствовать вашей задаче: классификация, регрессия и т.д.

5. **Проблема переобучения:**
   - Используйте отдельный тестовый набор данных для оценки модели после поиска гиперпараметров.
   - Никогда не оптимизируйте гиперпараметры на тестовом наборе.

6. **Параллелизация:**
   - Для ускорения используйте параметр `n_jobs=-1`, чтобы задействовать все ядра процессора.

### **Пример использования `GridSearchCV`**

#### Пример задачи: Оптимизация гиперпараметров для `RandomForestClassifier`

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Генерация данных
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Модель
model = RandomForestClassifier(random_state=42)

# Гиперпараметры для поиска
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',  # Метрика для оптимизации
    cv=5,  # 5-кратная кросс-валидация
    n_jobs=-1,  # Использовать все процессоры
    verbose=2
)

# Запуск поиска
grid_search.fit(X_train, y_train)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Оценка на тестовом наборе
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Точность на тестовом наборе:", accuracy_score(y_test, y_pred))

### **Результат выполнения `GridSearchCV`**
1. **`best_params_`:** Показывает комбинацию гиперпараметров, давшую наилучший результат.
2. **`best_score_`:** Среднее качество модели (по метрике `scoring`) на кросс-валидации с лучшими гиперпараметрами.
3. **`best_estimator_`:** Модель, обученная с лучшими гиперпараметрами.
4. **`cv_results_`:** Полная информация о всех комбинациях гиперпараметров.

### **Альтернативы `GridSearchCV`**

1. **`RandomizedSearchCV`:**
   - Вместо полного перебора случайно выбирает комбинации гиперпараметров.
   - Экономит время, особенно при большом количестве параметров.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Случайный поиск
param_distributions = {
   'n_estimators': randint(50, 200),
   'max_depth': [None, 10, 20, 30],
   'min_samples_split': [2, 5, 10],
   'min_samples_leaf': [1, 2, 4]
}

random_search = RandomizedSearchCV(
   estimator=model,
   param_distributions=param_distributions,
   n_iter=20,  # Количество случайных комбинаций
   scoring='accuracy',
   cv=5,
   n_jobs=-1,
   random_state=42,
   verbose=2
)

random_search.fit(X_train, y_train)
print("Лучшие параметры (RandomizedSearchCV):", random_search.best_params_)

2. **Bayesian Optimization (например, `Optuna`):**
   - Умный перебор параметров с использованием вероятностных моделей.
   - Быстрее, чем `GridSearchCV`, при этом результаты часто лучше.

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def objective(trial):
   n_estimators = trial.suggest_int('n_estimators', 50, 200)
   max_depth = trial.suggest_int('max_depth', 10, 30, log=True)
   min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
   min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)

   model = RandomForestClassifier(
       n_estimators=n_estimators,
       max_depth=max_depth,
       min_samples_split=min_samples_split,
       min_samples_leaf=min_samples_leaf,
       random_state=42
   )
   model.fit(X_train, y_train)
   y_pred = model.predict(X_test)
   return accuracy_score(y_test, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print("Лучшие параметры (Optuna):", study.best_params)

### **Выводы**
1. **GridSearchCV** хорошо работает для небольших задач, но может быть медленным при большом количестве гиперпараметров.
2. **`RandomizedSearchCV`** быстрее, но менее детален.
3. **Продвинутые методы (например, `Optuna`)** позволяют быстрее находить оптимальные параметры.
4. **Важно всегда иметь тестовый набор**, который не используется в процессе выбора гиперпараметров.

# Работа с интервалами гиперпараметров

Если гиперпараметры, найденные с помощью GridSearchCV, оказываются на границах заданных интервалов необходимо увеличить/изменить диапазон. Проверить влияние этих гиперпараметров на качество модели. Возможно, они не сильно влияют, и можно оставить их фиксированными.

В процессе использования `GridSearchCV`, если вы видите, что какой-то из гиперпараметров достигает границы интервала, это сигнал для уточнения его диапазона. При этом диапазоны остальных гиперпараметров, у которых значения находятся внутри интервала, можно оставить без изменений. Вот как это делается:

### **1. Уточняем диапазон только для "крайних" гиперпараметров**
Если один из гиперпараметров (например, `max_depth`) достигает верхней границы интервала, увеличиваем только его диапазон, оставляя другие неизменными.

#### Пример:

Допустим, вы провели поиск с таким диапазоном:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],  # "30" — крайнее значение
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3]
}

Результаты показали, что `max_depth=30` (крайнее значение), но другие гиперпараметры (`min_samples_split=5`, `min_samples_leaf=2`) находятся внутри интервалов. Вы можете расширить диапазон **только для `max_depth`**, например:

```python
param_grid = {
    'n_estimators': [50, 100, 150],  # Оставляем неизменным
    'max_depth': [20, 30, 40, 50],  # Увеличили диапазон
    'min_samples_split': [2, 5, 10],  # Оставляем неизменным
    'min_samples_leaf': [1, 2, 3]  # Оставляем неизменным
}
```

---
### **2. Сужаем диапазон, если найдено устойчивое значение**
Если по результатам `GridSearchCV` гиперпараметр стабильно оказывается, например, в середине диапазона, его можно зафиксировать или уменьшить диапазон для дальнейших оптимизаций. Это ускорит процесс подбора.

#### Пример:
Если после первого поиска `min_samples_leaf=2` везде показывает себя лучшим, диапазон можно зафиксировать:

```python
param_grid = {
    'n_estimators': [50, 100, 150],  
    'max_depth': [20, 30, 40],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [2]  # Фиксируем
}
```
---
### **3. Постепенная оптимизация**
Для сложных моделей и большого количества гиперпараметров (как в `XGBoost` или `RandomForest`) иногда разумно не оптимизировать все сразу, а поэтапно. Например:
- Сначала подбираете оптимальный диапазон для **важных параметров** (например, `max_depth` и `n_estimators`).
- Затем фиксируете их и уточняете остальные параметры.

---

### **4. Пример кода**

```python
# Первый этап: ищем max_depth
param_grid_stage_1 = {
    'n_estimators': [100],  # Фиксируем
    'max_depth': [10, 20, 30, 40, 50],  # Расширили диапазон
    'min_samples_split': [5],  # Фиксируем
    'min_samples_leaf': [2]  # Фиксируем
}

grid_search_stage_1 = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_stage_1,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)
grid_search_stage_1.fit(X_train, y_train)

print("Лучший max_depth:", grid_search_stage_1.best_params_['max_depth'])

# Второй этап: уточняем другие параметры
best_max_depth = grid_search_stage_1.best_params_['max_depth']

param_grid_stage_2 = {
    'n_estimators': [50, 100, 150],
    'max_depth': [best_max_depth],  # Фиксируем найденное значение
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3]
}

grid_search_stage_2 = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid_stage_2,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)
grid_search_stage_2.fit(X_train, y_train)

print("Лучшие параметры:", grid_search_stage_2.best_params_)
```

---

### **5. Итог**
1. **Расширяйте интервал только для тех гиперпараметров, которые достигают границ.**
2. Оставляйте диапазоны других параметров неизменными.
3. Используйте пошаговую оптимизацию для сложных моделей.
4. После нахождения оптимальных значений проверьте их на тестовой выборке, чтобы убедиться, что модель действительно улучшилась. 

### **Результат:**
Вы получите точные оптимальные значения гиперпараметров, избегая лишней вычислительной нагрузки.